In [1]:
!pip install selenium
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

root_path = "/content/drive/My Drive/Colab Notebooks/data"
os.mkdir(root_path) if not os.path.exists(root_path) else None

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
Mounted at /content/drive


2. Extraia informação a partir de uma página dinâmica à sua escolha (recorrentemente atualizada, por exemplo uma página que reúna notícias ou eventos) com recurso ao Selenium. Defina um job scheduler (pode recorrer a um time.sleep) que programe a obtenção dos dados a cada hora durante o espaço de 2 dias consecutivos. Guarde os resultados num ficheiro JSON. Registe o desenrolar do processo de obtenção de dados num ficheiro de logs (e.g., via biblioteca logging).

In [58]:
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json
import logging

def log(message, level=logging.INFO):
  logger = logging.error if level == logging.ERROR else logging.info
  logger(message)
  print(message)

def extract_data(element, key="time"):
  extract_text = lambda query: element.find_element(By.CSS_SELECTOR, query).text.strip()
  value = extract_text("h4.tempo" if key == "time" else "div.num")
  kicker = extract_text("label.kicker")
  title = extract_text(".titulo")
  return {
    key: value,
    "kicker": kicker,
    "title": title,
    "image": element.find_element(By.CSS_SELECTOR, "figure > img").get_attribute("src")
  }

def extract_news(base_url):
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=options)
  try:
    news_json = {
      "newest": {},
      "popular": {}
    }
    news_path = os.path.join(root_path, str(datetime.now())[:10] + ".json")
    if os.path.exists(news_path):
      with open(news_path, 'r', encoding='utf-8') as json_file:
        news_json = json.load(json_file)
    driver.get(base_url)
    time.sleep(5)
    log("Gathering latest news.")
    last_news = driver.find_elements(By.CSS_SELECTOR, 'div.minuto-item')
    for article in [extract_data(element) for element in last_news]:
      id = article["time"].replace(":", "") + article["kicker"].replace(" ", "") + article["title"].replace(" ", "")
      news_json["newest"][id] = article
    log("Gathering popular news.")
    popular_news = driver.find_elements(By.CSS_SELECTOR, "section.ml-hc article.box")
    news_json["popular"][str(datetime.now())[11:16]] = [extract_data(element, "num") for element in popular_news]
    with open(news_path, 'w', encoding='utf-8') as json_file:
      json.dump(news_json, json_file)
  except Exception as e:
    log("Error extracting news: " + str(e), logging.ERROR)
  finally:
    driver.quit()

base_url = "https://www.correio24horas.com.br/ultimas"
logging.basicConfig(filename=os.path.join(root_path, 'Mini Proj II - task2.log'),
                    level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')
start_time = datetime.now()
end_time = start_time + timedelta(hours=2)

while datetime.now() < end_time:
  log("Beginning a new data extraction.")
  extract_news(base_url)
  log("Extraction finished. Waiting for the next execution.\n")
  time.sleep(60)

Beginning a new data extraction.
Gathering latest news.
Gathering popular news.
Extraction finished. Waiting for the next execution.

Beginning a new data extraction.
Gathering latest news.
Gathering popular news.
Extraction finished. Waiting for the next execution.

Beginning a new data extraction.
Gathering latest news.
Gathering popular news.
Extraction finished. Waiting for the next execution.

Beginning a new data extraction.
Gathering latest news.
Gathering popular news.
Extraction finished. Waiting for the next execution.

Beginning a new data extraction.
Gathering latest news.
Gathering popular news.
Extraction finished. Waiting for the next execution.

Beginning a new data extraction.
Gathering latest news.
Gathering popular news.
Extraction finished. Waiting for the next execution.

Beginning a new data extraction.
Gathering latest news.
Gathering popular news.
Extraction finished. Waiting for the next execution.

Beginning a new data extraction.
Gathering latest news.
Gather

In [60]:
from IPython.display import HTML
import pandas

news_json = None
with open(os.path.join(root_path, str(datetime.now())[:10] + ".json"), 'r', encoding='utf-8') as json_file:
  news_json = json.load(json_file)

df = pandas.DataFrame(news_json["newest"].values())
pandas.set_option('display.max_colwidth', None)
image_formatter = lambda i: f'<img src="{i}" width="128px">'
print("Latest news:\n")
display(HTML(df.to_html(escape=False, formatters=dict(image=image_formatter))))
for key in news_json["popular"]:
  df = pandas.DataFrame(news_json["popular"][key])
  print(f"\nTop 10 most popular news at {key}:")
  display(HTML(df.to_html(escape=False, formatters=dict(image=image_formatter))))

Latest news:



,time,kicker,title,image
0,11:41,FOGO,Mãe e filho de 22 anos morrem em incêndio de casa no interior da Bahia,
1,11:26,VITÓRIA DO BRASIL,Atriz brasileira participa de episódio de 'Grey's Anatomy'; veja cena,
2,11:17,NOVIDADE,Bahia negocia com a Copa Airlines a retomada dos voos Cidade do Panamá-Salvador,
3,11:06,ZERO APROXIMAÇÃO,"Após rompimento de amizade, Rodriguinho revela mágoa de Salgadinho",
4,11:02,VIOLÊNCIA,Homem baleado em sequestro a ônibus no Rio recebe alta de UTI,
5,11:00,GASTRONOMIA,Páscoa de chef: confira 12 lugares com promoções e pratos especiais,
6,11:00,TUDO GRAÇAS À MÚSICA,Conheça a história da primeira violista baiana da Academia Filarmônica de Berlim,
7,11:00,SOM SALVADOR,Aprovado em 1° lugar no vestibular de Música: conheça trajetória de Tom Zé na Ufba,
8,11:00,CRISE,Embaixador da Venezuela no Brasil pede reunião com governo após críticas de Lula a Maduro,
9,12:00,SAÚDE,A batalha final: como a Bahia ficou livre da febre aftosa e deixará de usar vacina,



Top 10 most popular news at 14:43:


,num,kicker,title,image
0,01,PARA A HISTÓRIA,Final do Baianão terá clássico entre Bahia e Vitória pela 26ª vez; veja o retrospecto,
1,02,CRIME,Morto no Corredor da Vitória estava em liberdade há cinco meses,
2,03,ALÔ ALÔ,Saiba quais são os edifícios mais caros e luxuosos do Horto Florestal,
3,04,VEJA COMO FAZER,Desconto de até R$ 20 na fatura da Coelba termina no próximo domingo (31),
4,05,HOMEM-ARANHA,"Em novo curta, Miles Morales enfrenta crise de pânico e ansiedade",
5,06,PULOU MURO,PM que fugiu de prisão em Lauro é capturado em Feira de Santana,
6,07,PROBLEMAS ESTRUTURAIS,"Com irregularidades, dois armazéns da Codeba são interditados por auditores fiscais do trabalho",
7,08,SALVADOR,Deu arrasta-pé: calor não inibiu público de curtir 1° edição do Forró no Parque,
8,09,'CANTANO',Entenda como herança africana no sotaque fez o baiano ‘falar cantando’,
9,10,PRIVILÉGIOS,"Ivete Sangalo, Caetano Veloso, Nanda Costa, Adriana Esteves e mais: veja artistas que têm casa em Salvador",



Top 10 most popular news at 14:56:


...

,num,kicker,title,image
0,01,FOGO,Mãe e filho de 22 anos morrem em incêndio de casa no interior da Bahia,
1,02,PRIVILÉGIOS,"Ivete Sangalo, Caetano Veloso, Nanda Costa, Adriana Esteves e mais: veja artistas que têm casa em Salvador",
2,03,SALVADOR,Deu arrasta-pé: calor não inibiu público de curtir 1° edição do Forró no Parque,
3,04,SOM SALVADOR,Aprovado em 1º lugar no vestibular de Música: conheça trajetória de Tom Zé na Ufba,
4,05,PARA A HISTÓRIA,Final do Baianão terá clássico entre Bahia e Vitória pela 26ª vez; veja o retrospecto,
5,06,E.C. VITÓRIA,Vitória terá reforço no ataque e ao menos seis baixas no Ba-Vi das finais do Baiano,
6,07,BALADAS,Onde acontecem as festas não óbvias em Salvador o ano inteiro?,
7,08,CRIME,Morto no Corredor da Vitória estava em liberdade há cinco meses,
8,09,CADA VEZ MAIS ALTO,Cultura do Paredão se impõe com força e se consolida como fenômeno em Salvador,
9,10,ÁREA DE RISCO,Justiça determina interdição e desocupação imediata de casas em Morro de São Paulo,



Top 10 most popular news at 16:58:


,num,kicker,title,image
0,01,FOGO,Mãe e filho de 22 anos morrem em incêndio de casa no interior da Bahia,
1,02,PRIVILÉGIOS,"Ivete Sangalo, Caetano Veloso, Nanda Costa, Adriana Esteves e mais: veja artistas que têm casa em Salvador",
2,03,SALVADOR,Deu arrasta-pé: calor não inibiu público de curtir 1° edição do Forró no Parque,
3,04,SOM SALVADOR,Aprovado em 1º lugar no vestibular de Música: conheça trajetória de Tom Zé na Ufba,
4,05,PARA A HISTÓRIA,Final do Baianão terá clássico entre Bahia e Vitória pela 26ª vez; veja o retrospecto,
5,06,E.C. VITÓRIA,Vitória terá reforço no ataque e ao menos seis baixas no Ba-Vi das finais do Baiano,
6,07,BALADAS,Onde acontecem as festas não óbvias em Salvador o ano inteiro?,
7,08,CRIME,Morto no Corredor da Vitória estava em liberdade há cinco meses,
8,09,CADA VEZ MAIS ALTO,Cultura do Paredão se impõe com força e se consolida como fenômeno em Salvador,
9,10,ÁREA DE RISCO,Justiça determina interdição e desocupação imediata de casas em Morro de São Paulo,



Top 10 most popular news at 17:00:


,num,kicker,title,image
0,01,FOGO,Mãe e filho de 22 anos morrem em incêndio de casa no interior da Bahia,
1,02,PRIVILÉGIOS,"Ivete Sangalo, Caetano Veloso, Nanda Costa, Adriana Esteves e mais: veja artistas que têm casa em Salvador",
2,03,SALVADOR,Deu arrasta-pé: calor não inibiu público de curtir 1° edição do Forró no Parque,
3,04,SOM SALVADOR,Aprovado em 1º lugar no vestibular de Música: conheça trajetória de Tom Zé na Ufba,
4,05,PARA A HISTÓRIA,Final do Baianão terá clássico entre Bahia e Vitória pela 26ª vez; veja o retrospecto,
5,06,E.C. VITÓRIA,Vitória terá reforço no ataque e ao menos seis baixas no Ba-Vi das finais do Baiano,
6,07,BALADAS,Onde acontecem as festas não óbvias em Salvador o ano inteiro?,
7,08,CRIME,Morto no Corredor da Vitória estava em liberdade há cinco meses,
8,09,CADA VEZ MAIS ALTO,Cultura do Paredão se impõe com força e se consolida como fenômeno em Salvador,
9,10,ÁREA DE RISCO,Justiça determina interdição e desocupação imediata de casas em Morro de São Paulo,
